In [60]:
import xmltodict
import bokeh
import re
import datetime

In [184]:
# function definitions

# rex pattern to match the log file timestamp
ts_rexp = '^(\d{4}-\d{2}-\d{2})\s(\d{2}:\d{2}:\d{2})'

def parse_start_time(stdout):
    for line in so.splitlines():
        match = re.match(ts_rexp, so.splitlines()[0])
        if match is not None:
            return '{0}T{1}'.format(match[1], match[2])


def parse_times(test, setup=False):
    tdc_from = td_from = t_from = sc_from = s_from = sc_to = s_to = td_to = tdc_to = t_to = None
    if test.get('properties'):
        t_from = calc_time(test['properties']['property']['@value'])
        t_to = calc_time(test['properties']['property']['@value'], test['@time'])

    for line in test['system-out'].splitlines():
        if not setup:
            match = re.match(ts_rexp, line)
            t_from = calc_time('{0}T{1}'.format(match[1], match[2]))
        
        # let's search for 'SetupClass' messages
        if not sc_from:
            sc_match = re.match(ts_rexp + '[^$]*(Started setUpClass:)', line)
            if sc_match:
                sc_from = calc_time('{0}T{1}'.format(sc_match[1], sc_match[2]))
        if not s_from:
            s_match = re.match(ts_rexp + '[^$]*(Started setUp:)', line)
            if s_match:
                s_from = sc_to = calc_time('{0}T{1}'.format(s_match[1], s_match[2]))
        if not t_from:
            t_match = re.match(ts_rexp + '[^$]*(Started Test)', line)
            if t_match:
                t_from = calc_time('{0}T{1}'.format(t_match[1], t_match[2]))
                if s_from:
                    s_to = calc_time('{0}T{1}'.format(t_match[1], t_match[2]))
                else:
                    sc_to = calc_time('{0}T{1}'.format(t_match[1], t_match[2]))
        if not td_from:
            td_match = re.match(ts_rexp + '[^$]*(Started tearDown:)', line)
            if td_match:
                td_from = calc_time('{0}T{1}'.format(td_match[1], td_match[2]))
                t_to = td_from
        if not tdc_from:
            tdc_match = re.match(ts_rexp + '[^$]*(Started tearDownClass:)', line)
            if tdc_match:
                tdc_from = calc_time('{0}T{1}'.format(tdc_match[1], tdc_match[2]))
                if not t_to:
                    t_to = tdc_from
        return {
            'setupclass': {'from': sc_from, 'to': sc_to},
            'setup': {'from': s_from, 'to': s_to},
            'test': {'from': t_from, 'to': t_to},
            'teardown': {'from': td_from, 'to': td_to},
            'teardownclass': {'from': tdc_from, 'to': tdc_to},
        }

        
def calc_time(st, d=0):
    '''returns a datetime object representation of the passed datetime string with added duration time
    :param st: string in %Y-%m-%dT%H:%M:%S format
    :param d: string or number representing the duration in seconds
    '''
    s = datetime.datetime.strptime(st, '%Y-%m-%dT%H:%M:%S')
    return s+datetime.timedelta(0,float(d))

In [36]:
# load the xml file
with open('junits/tier1-parallel-results.xml', 'r') as f:
    t1p = xmltodict.parse(f.read())['testsuite']

In [186]:
testcases = [
    {'duration': parse_times(n),
     'name': n['@name'],
     'class': n['@classname'],
     
    } for n in t1p['testcase']] #if not n.get('properties')]

testcases


[{'duration': {'setupclass': {'from': datetime.datetime(2019, 10, 4, 15, 54, 54),
    'to': None},
   'setup': {'from': None, 'to': None},
   'test': {'from': datetime.datetime(2019, 10, 4, 15, 54, 54),
    'to': datetime.datetime(2019, 10, 4, 15, 55, 38, 204000)},
   'teardown': {'from': None, 'to': None},
   'teardownclass': {'from': None, 'to': None}},
  'name': 'test_positive_update_auto_attach',
  'class': 'tests.foreman.api.test_activationkey.ActivationKeyTestCase'},
 {'duration': {'setupclass': {'from': datetime.datetime(2019, 10, 4, 15, 54, 54),
    'to': None},
   'setup': {'from': None, 'to': None},
   'test': {'from': datetime.datetime(2019, 10, 4, 15, 54, 54),
    'to': datetime.datetime(2019, 10, 4, 15, 55, 38, 890000)},
   'teardown': {'from': None, 'to': None},
   'teardownclass': {'from': None, 'to': None}},
  'name': 'test_positive_delete',
  'class': 'tests.foreman.api.test_activationkey.ActivationKeyTestCase'},
 {'duration': {'setupclass': {'from': datetime.datetime(